<a target="_blank" href="https://colab.research.google.com/github/taobrienlbl/advanced_earth_science_data_analysis/blob/spring_2023_iub/lessons/04_digging_further_into_data_wrangling/04_estimating_building_height.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This exercise aims to build on your knowledge of the big-four data science libraries to use Kestrel atmospheric measurement devices to estimate the height difference between the ground floor and sixth floor of the Geology building.

# Background

Air pressure monotonically decreases with height in the atmosphere.  This is simply due to the fact that *air pressure is a measure of the weight of everything above*.  So higher up in the atmosphere there is less atmosphere (weight) above, and pressure decreases.

One of the basic principles of fluid dynamics is that a *pressure gradient*--a change in pressure over some distance--exerts a net force on the fluid that, in the absence of other forces, would cause the fluid to move.  So why then does the atmosphere not flow away to space in response to this pressure gradient?

To a very good approximation, the atmosphere is in *hydrostatic balance*, meaning that the vertical pressure gradient is balanced by the force of gravity on the air; gravity effectively negates the pressure gradient force.  This leads to one of the most essential equations in atmospheric science:

$$ \frac{\partial p}{\partial z} = -\rho g$$

where $p$ is the atmospheric pressure, $z$ is height away from the surface, $\rho$ is air density, and $g$ is the gravitational acceleration (we'll use $g \approx 9.806~\text{m s}^{-2}$ here).

If we combine this with the ideal gas law (written in a form that is convenient for atmospheric gasses, where $R_d \approx 287~\text{J kg}^{-1}\text{K}^{-1}$ represents a constant for air with the composition of Earth's atmosphere), we get a differential equation for pressure.

$$\text{Ideal gas law: }~p = \rho R_d T \to $$

$$ \frac{\partial p}{\partial z} = - \frac{p g}{R_d T} $$

We can approximate the derivatives in this equation using finite differences: $\partial p/\partial z \to \Delta p / \Delta z$, where $\Delta$ represents a (presumably small) difference between two pressure or height measurements.  We can then use this to solve for the change in height as a function of changes in pressure:

$$ \Delta z \approx - \frac{R_d T}{g} \frac{\Delta p}{p}$$

# Estimating height

Now imagine that we have a device that records pressure and temperature every second or so.  If you take that device and walk up a set of stairs, the recorded pressure will drop a bit for every second that you're ascending.  A pair of measurements taken at consecutive times can be used to estimate $\Delta p$, and the average of the two measurements can be used to estimate $p$ and $T$ over tha same time frame.  These values can be plugged in to the above formula to estimate the change in height between the two measurements.  The sum of all the changes in height then gives an estimate of the total change in height; we will have numerically integrated the differential equation that comes from combining hydrostatic balance and the ideal gas law.

## Instructions

1. form groups of two or three
1. acquaint yourself with the Kestrel measuring devices, and pair it to at least one of your phones using [Kestrel Link](https://kestrelinstruments.com/link-connectivity)
1. ensure that you can record measurements and export them to csv
1. take the kestrel device down to the ground floor of the Geology building; go to the loop
1. start recording
1. either walk up the stairwell, or take the elevator, up to the 7th floor (you can only access this floor from the western stairwell)
1. end recording
1. export the data to a CSV file and add it to your course git folder (into a folder for lesson 04)
1. estimate the difference in height between the two floors using the method above (*hint*: you should use `pandas`, and you should be able to make a new column for the calculation of $\Delta p$ and $\Delta z$ for each measurement, and then use the `.sum()` method)

# format
"FORMATTED DATE_TIME","Temperature","Wet Bulb Temp","Relative Humidity","Barometric Pressure","Altitude","Station Pressure","Wind Speed","Heat Index","Dew Point","Density Altitude","Crosswind","Headwind","Compass Magnetic Direction","Compass True Direction","Wind Chill","Data Type","Record name","Start time","Duration (H:M:S)","Location description","Location address","Location coordinates","Notes"
"YYYY-MM-DD HH:MM:SS","°F","°F","%","inHg","ft","inHg","mph","°F","°F","ft","mph","mph","Deg","Deg","°F"

In [102]:
""" Import data"""
import pandas as pd
import plotly.express as px

data = pd.read_csv('data.csv',skiprows=10 ,skipfooter=5 , names=["FORMATTED DATE_TIME","Temperature","Wet Bulb Temp","Relative Humidity","Barometric Pressure","Altitude","Station Pressure","Wind Speed","Heat Index","Dew Point","Density Altitude","Crosswind","Headwind","Compass Magnetic Direction","Compass True Direction","Wind Chill","Data Type"])
# Change InHg to pa
data['pressure Pa'] = data['Station Pressure']*3386.389
data['correct height (m)'] = data['Altitude']/3.281
data

/var/folders/qk/d2qw5j9x1h3cmpy31fr930ym0000gt/T/ipykernel_27385/2789477811.py:5: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.



,FORMATTED DATE_TIME,Temperature,Wet Bulb Temp,Relative Humidity,Barometric Pressure,Altitude,Station Pressure,Wind Speed,Heat Index,Dew Point,Density Altitude,Crosswind,Headwind,Compass Magnetic Direction,Compass True Direction,Wind Chill,Data Type,pressure Pa,correct height (m)
0,2023-09-15 02:52:34 PM,78.1,61.2,38.0,29.30,576,29.30,0.0,75.7,50.5,2101,--,--,--,--,78.1,session,99221.19770,175.556233
1,2023-09-15 02:52:39 PM,77.7,60.8,37.8,29.29,577,29.30,0.0,75.0,50.0,2076,--,--,--,--,77.5,session,99221.19770,175.861018
2,2023-09-15 02:52:44 PM,77.9,61.2,38.3,29.29,581,29.29,0.0,75.6,50.5,2094,--,--,--,--,77.9,session,99187.33381,177.080158
3,2023-09-15 02:52:49 PM,76.8,60.4,38.7,29.29,581,29.29,0.0,74.5,49.8,2024,--,--,--,--,76.8,session,99187.33381,177.080158
4,2023-09-15 02:53:18 PM,75.3,58.4,38.5,29.29,577,29.29,0.0,72.0,47.4,1851,--,--,--,--,74.3,session,99187.33381,175.861018
5,2023-09-15 02:53:19 PM,71.7,57.5,42.1,29.28,585,29.29,0.0,70.2,47.4,1691,--,--,--,--,71.6,session,99187.33381,178.299299
6,2023-09-15 02:53:24 PM,72.0,58.1,43.5,29.28,585,29.28,0.0,70.7,48.6,1723,--,--,--,--,72.0,session,99153.46992,178.299299
7,2023-09-15 02:53:29 PM,72.4,59.0,45.2,29.28,592,29.28,0.0,71.1,50.0,1762,--,--,--,--,72.3,session,99153.46992,180.432795
8,2023-09-15 02:53:34 PM,72.8,59.5,46.3,29.28,589,29.28,0.0,71.4,51.0,1789,--,--,--,--,72.7,session,99153.46992,179.518440
9,2023-09-15 02:53:40 PM,73.1,59.9,46.8,29.28,589,29.28,0.0,71.8,51.6,1819,--,--,--,--,73.0,session,99153.46992,179.518440


In [103]:
fig = px.line(data, x="FORMATTED DATE_TIME", y="pressure Pa")
fig.show()

In [104]:
""" Calculate Pressure Change by min and max """

data['pressure change'] = data['pressure Pa'].diff()
data.loc[0,'pressure change'] = 0

# Calculate average press change
data['avg_press'] = (data['pressure Pa'] + data['pressure Pa'].shift(1))/2
data.loc[0,'avg_press'] = data.loc[0,'pressure Pa']

# convert F to k, average
data['tem_k'] = (data['Temperature']-32)*5/9 +273
data['avg_tem_k'] = (data['tem_k'] + data['tem_k'].shift(1))/2
data.loc[0,'avg_tem_k'] = data.loc[0,'tem_k']
data

,FORMATTED DATE_TIME,Temperature,Wet Bulb Temp,Relative Humidity,Barometric Pressure,Altitude,Station Pressure,Wind Speed,Heat Index,Dew Point,...,Compass Magnetic Direction,Compass True Direction,Wind Chill,Data Type,pressure Pa,correct height (m),pressure change,avg_press,tem_k,avg_tem_k
0,2023-09-15 02:52:34 PM,78.1,61.2,38.0,29.30,576,29.30,0.0,75.7,50.5,...,--,--,78.1,session,99221.19770,175.556233,0.00000,99221.197700,298.611111,298.611111
1,2023-09-15 02:52:39 PM,77.7,60.8,37.8,29.29,577,29.30,0.0,75.0,50.0,...,--,--,77.5,session,99221.19770,175.861018,0.00000,99221.197700,298.388889,298.500000
2,2023-09-15 02:52:44 PM,77.9,61.2,38.3,29.29,581,29.29,0.0,75.6,50.5,...,--,--,77.9,session,99187.33381,177.080158,-33.86389,99204.265755,298.500000,298.444444
3,2023-09-15 02:52:49 PM,76.8,60.4,38.7,29.29,581,29.29,0.0,74.5,49.8,...,--,--,76.8,session,99187.33381,177.080158,0.00000,99187.333810,297.888889,298.194444
4,2023-09-15 02:53:18 PM,75.3,58.4,38.5,29.29,577,29.29,0.0,72.0,47.4,...,--,--,74.3,session,99187.33381,175.861018,0.00000,99187.333810,297.055556,297.472222
5,2023-09-15 02:53:19 PM,71.7,57.5,42.1,29.28,585,29.29,0.0,70.2,47.4,...,--,--,71.6,session,99187.33381,178.299299,0.00000,99187.333810,295.055556,296.055556
6,2023-09-15 02:53:24 PM,72.0,58.1,43.5,29.28,585,29.28,0.0,70.7,48.6,...,--,--,72.0,session,99153.46992,178.299299,-33.86389,99170.401865,295.222222,295.138889
7,2023-09-15 02:53:29 PM,72.4,59.0,45.2,29.28,592,29.28,0.0,71.1,50.0,...,--,--,72.3,session,99153.46992,180.432795,0.00000,99153.469920,295.444444,295.333333
8,2023-09-15 02:53:34 PM,72.8,59.5,46.3,29.28,589,29.28,0.0,71.4,51.0,...,--,--,72.7,session,99153.46992,179.518440,0.00000,99153.469920,295.666667,295.555556
9,2023-09-15 02:53:40 PM,73.1,59.9,46.8,29.28,589,29.28,0.0,71.8,51.6,...,--,--,73.0,session,99153.46992,179.518440,0.00000,99153.469920,295.833333,295.750000


In [105]:
""" Diff hight in each data point """

# set constants
Rd = 287 # J/kg/K
g = 9.806 # m/s^2

# calculate by formula
data['height'] = -Rd*data['avg_tem_k']*data['pressure change']/(g*data['avg_press'])
data['accu_height'] = data['height'].cumsum()
data['accu correct height (m)'] = data['correct height (m)'] - data.loc[0,'correct height (m)']
data

,FORMATTED DATE_TIME,Temperature,Wet Bulb Temp,Relative Humidity,Barometric Pressure,Altitude,Station Pressure,Wind Speed,Heat Index,Dew Point,...,Data Type,pressure Pa,correct height (m),pressure change,avg_press,tem_k,avg_tem_k,height,accu_height,accu correct height (m)
0,2023-09-15 02:52:34 PM,78.1,61.2,38.0,29.30,576,29.30,0.0,75.7,50.5,...,session,99221.19770,175.556233,0.00000,99221.197700,298.611111,298.611111,-0.000000,-0.000000,0.000000
1,2023-09-15 02:52:39 PM,77.7,60.8,37.8,29.29,577,29.30,0.0,75.0,50.0,...,session,99221.19770,175.861018,0.00000,99221.197700,298.388889,298.500000,-0.000000,-0.000000,0.304785
2,2023-09-15 02:52:44 PM,77.9,61.2,38.3,29.29,581,29.29,0.0,75.6,50.5,...,session,99187.33381,177.080158,-33.86389,99204.265755,298.500000,298.444444,2.981673,2.981673,1.523926
3,2023-09-15 02:52:49 PM,76.8,60.4,38.7,29.29,581,29.29,0.0,74.5,49.8,...,session,99187.33381,177.080158,0.00000,99187.333810,297.888889,298.194444,-0.000000,2.981673,1.523926
4,2023-09-15 02:53:18 PM,75.3,58.4,38.5,29.29,577,29.29,0.0,72.0,47.4,...,session,99187.33381,175.861018,0.00000,99187.333810,297.055556,297.472222,-0.000000,2.981673,0.304785
5,2023-09-15 02:53:19 PM,71.7,57.5,42.1,29.28,585,29.29,0.0,70.2,47.4,...,session,99187.33381,178.299299,0.00000,99187.333810,295.055556,296.055556,-0.000000,2.981673,2.743066
6,2023-09-15 02:53:24 PM,72.0,58.1,43.5,29.28,585,29.28,0.0,70.7,48.6,...,session,99153.46992,178.299299,-33.86389,99170.401865,295.222222,295.138889,2.949655,5.931328,2.743066
7,2023-09-15 02:53:29 PM,72.4,59.0,45.2,29.28,592,29.28,0.0,71.1,50.0,...,session,99153.46992,180.432795,0.00000,99153.469920,295.444444,295.333333,-0.000000,5.931328,4.876562
8,2023-09-15 02:53:34 PM,72.8,59.5,46.3,29.28,589,29.28,0.0,71.4,51.0,...,session,99153.46992,179.518440,0.00000,99153.469920,295.666667,295.555556,-0.000000,5.931328,3.962207
9,2023-09-15 02:53:40 PM,73.1,59.9,46.8,29.28,589,29.28,0.0,71.8,51.6,...,session,99153.46992,179.518440,0.00000,99153.469920,295.833333,295.750000,-0.000000,5.931328,3.962207


In [106]:
fig = px.line(data, x="FORMATTED DATE_TIME", y=data.columns[24:26])
fig.show()